## Importing libraires

In [1]:
import numpy as np
import string

## Function for removing puntuation

In [2]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

## Initial state, First order and Second order transition matrix

In [3]:
initial = {}
first_order = {}
second_order = {}

In [4]:
def add2dict(d, k, v):
    if k not in d:
        d[k] = {}
    d[k][v] = d[k].get(v, 0) +1

In [5]:
vocabulary = []
for line in open('datasets/robert_frost.txt'):
    tokens = remove_punctuation(line.rstrip().lower()).split()

    for i in range(len(tokens)):
        t = tokens[i]
        if t not in vocabulary:
            vocabulary.append(t)

        if i == 0:
            initial[t] = initial.get(t, 0.) + 1 
        else:
            t_1 = tokens[i-1]
            if i == len(tokens) - 1:
                add2dict(second_order, (t_1, t), 'stop')
            if i == 1:
                add2dict(first_order, t_1, t)
            else:
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

In [6]:
# Normalizing
def normalize(d):
    total = sum(d.values())
    for k, v in d.items():
        d[k] = v/total

In [7]:
normalize(initial)

In [8]:
for k, v in first_order.items():
    normalize(v)

In [9]:
for k, v in second_order.items():
    normalize(v)

## Size of vocabulary

In [22]:
V = len(vocabulary)
print(f'Total vocabulary in the poem (V): {V}')

Total vocabulary in the poem (V): 2197


In [11]:
# pi has shape V
i_size = len(initial)
print(f'Size of initial state is: {i_size}')
print(f'Total value stored: {i_size/V*100:.4f}%')

Size of initial state is: 305
Total value stored: 13.8826%


In [12]:
fo_size = 0
for k, v in first_order.items():
    fo_size += len(v)

print(f'Size of first order transition matrix is: {fo_size}')
print(f'Total value stored: {i_size/(V*V)*100:.4f}%')

Size of first order transition matrix is: 1195
Total value stored: 0.0063%


In [13]:
so_size = 0
for k, v in second_order.items():
    so_size += len(v)

print(f'Size of second order transition matrix is: {so_size}')
print(f'Total value stored: {i_size/(V*V*V)*100:.6f}%')

Size of second order transition matrix is: 8854
Total value stored: 0.000003%


## Generate function

In [14]:
def sample_word(d):
  p0 = np.random.random()

  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t
  assert(False)

In [15]:
def generate():
  for i in range(4): # generate 4 lines
    sentence = []

    # initial word
    w0 = sample_word(initial)
    sentence.append(w0)

    # sample second word
    w1 = sample_word(first_order[w0])
    sentence.append(w1)

    # second-order transitions until END
    while True:
        w2 = sample_word(second_order[(w0, w1)])
        if w2 == 'stop':
            break
        sentence.append(w2)
        w0 = w1
        w1 = w2
    print(' '.join(sentence))

In [21]:
generate()

oh if youre lost enough to find out what to do right
john hall touch me not if he knows hes kinder than the cellar
at having eased her heart of one that didnt thrive
one and one other yes for there were no cottages found


In [17]:
# Exercise:
#
# Determine the vocabulary size (V)
# We know that pi has shape V, A1 has shape V x V, and A2 has shape V x V x V
#
# In comparison, how many values are stored in our dictionaries?

In [18]:
# Exercise 2:
# We can skip the step where we accumulate all the possible next words in a list
# E.g. [cat, cat, dog, dog, dog, ...]
#
# Instead, like we do with the initial state distribution, create the dictionary
# of counts directly as you loop through the data.
#
# You'll no longer need list2pdict()